In [327]:
from astropy.io import fits
from astropy import wcs
import bz2
import os
import urllib.request
import numpy as np
import copy
import matplotlib

In [394]:
def asymmetry(ra, dec, r, petrorad, expab, expphi, run, camcol, field, rowf, colf, FITS_directory='Fits files/'  ,rerun=301):
    #Returns an array of the fourier A1 parameter and the phase angle for a given object in the SDSS field specified
    #by run, rerun, camcol, field, row, col @ row, col.
    #r= Radius (in number of petrosian radiuses) to calculate the fourier coefficient for
    #petrorad=the petrosian radius of the object in arcseconds
    #expphi = The angle of the exponential fit, expressed as degrees, N through E
    #expAB= The ratio of the semi major axes of the disk
    # Run, rerun, camcol, field = Specified the SDSS field the object is contained in
    # rowf, colf = the pixel coordinates of the centre of the image, in FITS coordinates (ie, first pixel is 1, not zero) 
    # FITS_directory=Location of folder of fits files
    #Open FITS file from disk
    try:
        #Generate the name the FITS file will be stored under
        runstring='0'*(6-len(str(run)))+str(run) 
        fieldstring='0'*(4-len(str(field)))+str(field)
        fitsname='frame-i-'+runstring+'-'+str(camcol)+'-'+fieldstring
        #Try to open the fits file. If it throws a filenotfound error, it's because this field hasn't been downloaded yet
        file=fits.open(FITS_directory+fitsname+'.fits')
    except FileNotFoundError:
        #Generate the URL the zipped FITS file will be stored under
        fitsurl='http://data.sdss3.org/sas/dr12/boss/photoObj/frames/'+str(rerun)+'/'+str(run)+'/'+str(camcol)+'/'+fitsname+'.fits.bz2'
        #Save the zipped FITS file, extract the unzipped fits file
        urllib.request.urlretrieve(fitsurl, FITS_directory+fitsname+'.fits.bz2')
        f=bz2.open(FITS_directory+fitsname+'.fits.bz2')
        #Save the unzipped FITS file to disk
        save=fits.open(f)
        save.writeto(FITS_directory+fitsname+'.fits')
        save.close()
        #Remove the zipped file to save space
        os.remove(FITS_directory+fitsname+'.fits.bz2')
        file=fits.open(FITS_directory+fitsname+'.fits')
    image=file[0] 
    data=image.data

    #Convert centre row and column to python convention (starting at zero), instead of FITS convention(starting at 1)
    rowc=rowf-1
    colc=colf-1
    
    
    #Get coordinate parameters from header using astropy WCS class. 
    coords=wcs.WCS(image.header)
    
    #Checks ra & dec agree with FITS header to within 1 pixel. 
    checkradec(ra, dec, coords, rowc, colc)
    
    #Set up sub-function to return row, col pairs for given ra, dec values
    def pix(raf,decf):
        #Takes values of ra and dec (in degrees). Converts to an array of [row, col] in pixel values
        array=coords.all_world2pix(raf,decf,0)
        return array[::-1]
    

    #Get length of semi-major and semi-minor axes of petrosian ellipse. Assume petrosian radius is the geometric mean of a & b. In degrees
    a=(2./(1+expab**2))**0.5*petrorad/3600.
    b=(2./(1+expab**-2))**0.5*petrorad/3600.
    
    #Get semi-major and semi-minor axes of the ellipse to calculate the fourier coefficients around. In arcsec
    ar=a*r
    br=b*r
    
    
    #Create an array of theta values to sample the ellipse at
    dtheta=360./360.
    thetas=np.arange(0,360,dtheta)
    thetarad=[i*2*np.pi/360. for i in thetas]
    rs=getrs(thetas, ar, br, expphi)
    
    #Create array with data values for each theta, r combination
    pixbrightness=[]
    for i in range(len(thetas)):
        #Calculate ra, dec values of target
        #First calculate distances from centre
        drai=rs[i]*np.sin(thetarad[i])
        ddeci=rs[i]*np.cos(thetarad[i])
        #Then add on coordinates of centre
        rai=ra+drai
        deci=dec+ddeci
        
        #Convert to pixels
        pixel=pix(rai,deci)
        rowi=float(pixel[0])
        coli=float(pixel[1])
        rowiround=int(round(rowi))
        coliround=int(round(coli))
        pixbrightness.append(data[rowiround, coliround])
        
        
    #Weight the pixel brightnesses by cos(theta) and sin(theta)
    brightnesssin=[pixbrightness[i]*np.sin(thetarad[i]) for i in range(len(thetarad))]
    brightnesscos=[pixbrightness[i]*np.cos(thetarad[i]) for i in range(len(thetarad))]
    #Find the a1 & b1 fourier coefficients. 1/pi * integral over pixelbrighness sin/cos (integrated using np.trapz). f(x)=ancosntheta+ bnsinntheta, summed over all n
    a1=np.pi**-1*np.trapz(brightnesscos, thetarad)
    b1=np.pi**-1*np.trapz(brightnesssin, thetarad)
    a0=np.pi**-1*np.trapz(pixbrightness, thetarad)
    #Now find fourier coefficients in the form f(x)=Ancos(ntheta-phi)
    A1=(a1**2+b1**2)**0.5
    phi=np.arctan(-a1/b1)
    return [A1, phi*360./2/np.pi, A1/a0, a0]
    file.close()
    
    
    
def ellipse(a,b,theta,phi):
    #returns the r coordinate of the ellipse with:
    #semi major axes a,b at polar angle theta degrees (measured from north, through east),
    #with semi-major axis offset from north by phi degrees (measured from north through east)
    thetarad=theta*2*np.pi/360.
    phirad=phi*2*np.pi/360.
    r=a*b/((b*np.cos(thetarad-phirad))**2+(a*np.sin(thetarad-phirad))**2)**0.5
    return r

def checkradec(ra, dec, coords, row, col):
    #checks to see if the pixel coordinates returned by a WCS object coords match ra and dec to correct values row and column
    reportedrow=coords.all_world2pix(ra,dec,0)[1]
    reportedcol=coords.all_world2pix(ra,dec,0)[0]
    try:
        if (reportedcol-col)**2>1 or (reportedrow-row)**2>1:
            raise ValueError('Coordinates from PhotoObj and ra & dec values do not match')
    except ValueError:
        raise
def getrs(thetas, a, b, phi):
    rs=[]
    for i in thetas:
        rs.append(ellipse(a,b,i,phi))
    return rs
def functionchecker(ra, dec, r, petrorad, expab, expphi, run, camcol, field, rowf, colf, A1, A0, phi, FITS_directory='Fits files/'  ,rerun=301, A2=0.2, A3=0.14):
    #Opens the fits file for the given galaxy coordinates, then edits it to make the specific fourier components A1 and phi at radius r petrorads
       #Creates a fits file with an ellipse drawn on at r petrosian radii for a given object in the SDSS field specified
    #by run, rerun, camcol, field, row, col @ row, col.
    #r= Radius (in number of petrosian radiuses) to calculate the fourier coefficient for
    #petrorad=the petrosian radius of the object in arcseconds
    #expphi = The angle of the exponential fit, expressed as degrees, N through E
    #expAB= The ratio of the semi major axes of the disk
    # Run, rerun, camcol, field = Specified the SDSS field the object is contained in
    # rowf, colf = the pixel coordinates of the centre of the image, in FITS coordinates (ie, first pixel is 1, not zero) 
    #A1, A0, phi(degrees) - first fourier coefficients
    # FITS_directory=Location of folder of fits files
    #Open FITS file from disk
    try:
        #Generate the name the FITS file will be stored under
        runstring='0'*(6-len(str(run)))+str(run) 
        fieldstring='0'*(4-len(str(field)))+str(field)
        fitsname='frame-i-'+runstring+'-'+str(camcol)+'-'+fieldstring
        #Try to open the fits file. If it throws a filenotfound error, it's because this field hasn't been downloaded yet
        file=fits.open(FITS_directory+fitsname+'.fits')
    except FileNotFoundError:
        #Generate the URL the zipped FITS file will be stored under
        fitsurl='http://data.sdss3.org/sas/dr12/boss/photoObj/frames/'+str(rerun)+'/'+str(run)+'/'+str(camcol)+'/'+fitsname+'.fits.bz2'
        #Save the zipped FITS file, extract the unzipped fits file
        urllib.request.urlretrieve(fitsurl, FITS_directory+fitsname+'.fits.bz2')
        f=bz2.open(FITS_directory+fitsname+'.fits.bz2')
        #Save the unzipped FITS file to disk
        save=fits.open(f)
        save.writeto(FITS_directory+fitsname+'.fits')
        save.close()
        #Remove the zipped file to save space
        os.remove(FITS_directory+fitsname+'.fits.bz2')
        file=fits.open(FITS_directory+fitsname+'.fits')
    image=file[0]
    data=image.data
    
    #Convert centre row and column to python convention (starting at zero), instead of FITS convention(starting at 1)
    rowc=rowf-1
    colc=colf-1
    
    
    #Get coordinate parameters from header using astropy WCS class. 
    coords=wcs.WCS(image.header)
    
    #Checks ra & dec agree with FITS header to within 1 pixel. 
    checkradec(ra, dec, coords, rowc, colc)
    
    #Set up sub-function to return row, col pairs for given ra, dec values
    def pix(raf,decf):
        #Takes values of ra and dec (in degrees). Converts to an array of [row, col] in pixel values
        array=coords.all_world2pix(raf,decf,0)
        return array[::-1]
    

    #Get length of semi-major and semi-minor axes of petrosian ellipse. Assume petrosian radius is the geometric mean of a & b. In degrees
    a=(2./(1+expab**2))**0.5*petrorad/3600.
    b=(2./(1+expab**-2))**0.5*petrorad/3600.
    
    #Get semi-major and semi-minor axes of the ellipse to calculate the fourier coefficients around. In arcsec
    ar=a*r
    br=b*r
    
    
    #Create an array of theta values to sample the ellipse at
    dtheta=360./360.
    thetas=np.arange(0,360,dtheta)
    rs=getrs(thetas, ar, br, expphi)
    
    def fourier(thetar, An, phi, n):
        return An*np.sin(n*(thetar-phi*2*np.pi/360))
    #Change value of each pixel in ellipse in ellipselayer
    for i in range(len(thetas)):
        thetarad=thetas[i]*2*np.pi/360.
        #Calculate ra, dec values of target
        #First calculate distances from centre
        drai=rs[i]*np.sin(thetarad)
        ddeci=rs[i]*np.cos(thetarad)
        #Then add on coordinates of centre
        rai=ra+drai
        deci=dec+ddeci
        
        #Convert to pixels
        pixel=pix(rai,deci)
        rowi=float(pixel[0])
        coli=float(pixel[1])
        rowiround=int(round(rowi))
        coliround=int(round(coli))
        #Sets corresponding pixel to value 2
        data[rowiround,coliround]=fourier(thetarad,A1,phi,1)+fourier(thetarad,A2,phi,2)+fourier(thetarad,A3,phi,3)+A0/2.
    #Adds the layer to the original fits file and saves as a new fits file with camcol G (for generated) (to differentiate between real and generated fits files)
    newfilename='frame-i-'+runstring+'-G'+'-'+fieldstring
    file.writeto(FITS_directory+newfilename+'.fits')
   
    file.close()

In [399]:
print(asymmetry(211.6933,-0.7880827,0.3,11.10158,0.7288934,67.95884,752,2,455,855.3524,543.3588))
#functionchecker(211.6933,-0.7880827,0.5,11.10158,0.7288934,67.95884,752,2,455,855.3524,543.3588, 0.2,0.34,67.95)
asymmetry(211.6933,-0.7880827,0.5,11.10158,0.7288934,67.95884,752,'G',455,855.3524,543.3588)
asymmetry(ra=183.1636,dec=-2.72125371210762,r=1,petrorad=5.679752,expab=0.6782587,expphi=15.41315,run=1231, camcol=3, field=93, rowf=575.2551, colf=1835.075)


the RADECSYS keyword is deprecated, use RADESYSa. [astropy.wcs.wcs]


[0.012576442728398128, 15.974719979139454, 0.023744170317336451, 0.52966444227430542]


[0.023296133095775235,
 -25.837470722475,
 0.02385941568781921,
 0.97639160156249993]

In [367]:
def plotellipse(ra, dec, r, petrorad, expab, expphi, run, camcol, field, rowf, colf, FITS_directory='Fits files/'  ,rerun=301):
    #Creates a fits file with an ellipse drawn on at r petrosian radii for a given object in the SDSS field specified
    #by run, rerun, camcol, field, row, col @ row, col.
    #r= Radius (in number of petrosian radiuses) to calculate the fourier coefficient for
    #petrorad=the petrosian radius of the object in arcseconds
    #expphi = The angle of the exponential fit, expressed as degrees, N through E
    #expAB= The ratio of the semi major axes of the disk
    # Run, rerun, camcol, field = Specified the SDSS field the object is contained in
    # rowf, colf = the pixel coordinates of the centre of the image, in FITS coordinates (ie, first pixel is 1, not zero) 
    # FITS_directory=Location of folder of fits files
    #Open FITS file from disk
    try:
        #Generate the name the FITS file will be stored under
        runstring='0'*(6-len(str(run)))+str(run) 
        fieldstring='0'*(4-len(str(field)))+str(field)
        fitsname='frame-i-'+runstring+'-'+str(camcol)+'-'+fieldstring
        #Try to open the fits file. If it throws a filenotfound error, it's because this field hasn't been downloaded yet
        file=fits.open(FITS_directory+fitsname+'.fits')
    except FileNotFoundError:
        #Generate the URL the zipped FITS file will be stored under
        fitsurl='http://data.sdss3.org/sas/dr12/boss/photoObj/frames/'+str(rerun)+'/'+str(run)+'/'+str(camcol)+'/'+fitsname+'.fits.bz2'
        #Save the zipped FITS file, extract the unzipped fits file
        urllib.request.urlretrieve(fitsurl, FITS_directory+fitsname+'.fits.bz2')
        f=bz2.open(FITS_directory+fitsname+'.fits.bz2')
        #Save the unzipped FITS file to disk
        save=fits.open(f)
        save.writeto(FITS_directory+fitsname+'.fits')
        save.close()
        #Remove the zipped file to save space
        os.remove(FITS_directory+fitsname+'.fits.bz2')
        file=fits.open(FITS_directory+fitsname+'.fits')
    image=file[0]
    data=image.data
    #Set up new layer with same shape as image data, with value -100 in all pixels
    ellipselayer=copy.copy(file[0])
    ellipselayer.data=np.full_like(ellipselayer.data,-100.)
    

    #Convert centre row and column to python convention (starting at zero), instead of FITS convention(starting at 1)
    rowc=rowf-1
    colc=colf-1
    
    
    #Get coordinate parameters from header using astropy WCS class. 
    coords=wcs.WCS(image.header)
    
    #Checks ra & dec agree with FITS header to within 1 pixel. 
    checkradec(ra, dec, coords, rowc, colc)
    
    #Set up sub-function to return row, col pairs for given ra, dec values
    def pix(raf,decf):
        #Takes values of ra and dec (in degrees). Converts to an array of [row, col] in pixel values
        array=coords.all_world2pix(raf,decf,0)
        return array[::-1]
    

    #Get length of semi-major and semi-minor axes of petrosian ellipse. Assume petrosian radius is the geometric mean of a & b. In degrees
    a=(2./(1+expab**2))**0.5*petrorad/3600.
    b=(2./(1+expab**-2))**0.5*petrorad/3600.
    
    #Get semi-major and semi-minor axes of the ellipse to calculate the fourier coefficients around. In arcsec
    ar=a*r
    br=b*r
    
    
    #Create an array of theta values to sample the ellipse at
    dtheta=360./360.
    thetas=np.arange(0,360,dtheta)
    rs=getrs(thetas, ar, br, expphi)
    
    #Change value of each pixel in ellipse in ellipselayer
    for i in range(len(thetas)):
        thetarad=thetas[i]*2*np.pi/360.
        #Calculate ra, dec values of target
        #First calculate distances from centre
        drai=rs[i]*np.sin(thetarad)
        ddeci=rs[i]*np.cos(thetarad)
        #Then add on coordinates of centre
        rai=ra+drai
        deci=dec+ddeci
        
        #Convert to pixels
        pixel=pix(rai,deci)
        rowi=float(pixel[0])
        coli=float(pixel[1])
        rowiround=int(round(rowi))
        coliround=int(round(coli))
        #Sets corresponding pixel to value 2
        ellipselayer.data[rowiround,coliround]=2.
        data[rowiround,coliround]=2.
    #Adds the layer to the original fits file and saves as a new fits file
    file.append(ellipselayer)
    file.writeto(FITS_directory+fitsname+'_with_ellipse_at_'+str(r)+'_petrorad'+'.fits')
   
    file.close()

In [339]:
plotellipse(211.6933,-0.7880827,1,11.10158,0.7288934,67.95884,752,2,455,855.3524,543.3588)

Filename: Fits files/frame-i-000752-2-0455.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU      86   (2048, 1489)   float32   
  1                1 ImageHDU         6   (2048,)   float32   
  2                1 BinTableHDU     27   1R x 3C   [49152E, 2048E, 1489E]   
  3                1 BinTableHDU     79   1R x 31C   [J, 3A, J, A, D, D, 2J, J, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, E, E]   
  4                1 ImageHDU        85   (2048, 1489)   float32   
None


the RADECSYS keyword is deprecated, use RADESYSa. [astropy.wcs.wcs]
